# General setup

In [2]:
import sympy as sym

In [3]:
x, y = sym.symbols('x y', real=True)

In [28]:
h = sym.Symbol('hbar')
omega1 = sym.Symbol('omega1', real=True)
omega2 = sym.Symbol('omega2', real=True)
n1 = sym.Symbol('n1')
n2 = sym.Symbol('n2')

In [43]:
e = sym.exp
t = sym.Symbol('t', real=True)
c = sym.Symbol('c', real=True)

In [44]:
E_n1n2 = h*( (sym.Rational(1/2) + n1)*omega1 + (sym.Rational(1/2) + n2)*omega2 )
E_n1n2

hbar*(omega1*(n1 + 1/2) + omega2*(n2 + 1/2))

In [45]:
E_n1n2 = E_n1n2.subs(h,1)
E_n1n2

omega1*(n1 + 1/2) + omega2*(n2 + 1/2)

Eigenfunctions 

In [46]:
def psi_n1n2(n1_val, n2_val):
    C_n1n2_numerator = (omega1*omega2)**(sym.Rational(1/4))
    C_n1n2_divider = sym.sqrt(sym.pi* sym.factorial(n1) * sym.factorial(n2)* 2**(n1+n2))
    C_n1n2 = C_n1n2_numerator/C_n1n2_divider
    H_n1 = sym.hermite_poly(n1_val, sym.sqrt(omega1)*x)
    H_n2 = sym.hermite_poly(n2_val, sym.sqrt(omega2)*y)
    expfunc1 = sym.exp(-sym.I*E_n1n2*t)
    expfunc2 = sym.exp(-sym.Rational(1/2)*(omega1*x**2 + omega2*y**2))
    product = C_n1n2*expfunc1*expfunc2
    psi_n1n2 = product.subs([(n1, n1_val),(n2, n2_val)])*H_n1*H_n2
    return psi_n1n2

Wave function

In [71]:
c1, c2 = sym.symbols('c1 c2', real=True)

$\Psi(x,y,t) = \psi_{0,0} + c_1 \psi_{1,0} + c_2 \psi_{1,1}$

In [124]:
Psi = psi_n1n2(0,0) + c1*psi_n1n2(1,0) + c2*psi_n1n2(1,1)

# Compute nodal points

In [125]:
Psi = Psi.subs(omega1,1)
Psi = Psi.subs(omega2,1/c**2)

In [126]:
Psi = Psi.factor()
Psi

(sqrt(2)*c1*x*exp(I*t/c**2)*Abs(c) + 2*c2*x*y + exp(I*t)*exp(I*t/c**2)*Abs(c))*exp(-x**2/2)*exp(-3*I*t/2)*exp(-y**2/(2*c**2))*exp(-3*I*t/(2*c**2))/(sqrt(pi)*Abs(c)**(3/2))

Extract LHS factor

In [75]:
factor_main = Psi.args[2]
factor_main

sqrt(2)*c1*x*exp(I*t/c**2)*Abs(c) + 2*c2*x*y + exp(I*t)*exp(I*t/c**2)*Abs(c)

Rewrite $e^{it}$ using polar form

In [76]:
expr = factor_main
expr = expr.rewrite(sym.cos).expand()
expr = expr.simplify()
expr

sqrt(2)*I*c1*x*sin(t/c**2)*Abs(c) + sqrt(2)*c1*x*cos(t/c**2)*Abs(c) + 2*c2*x*y + I*sin(t + t/c**2)*Abs(c) + cos(t + t/c**2)*Abs(c)

Extract REAL part

In [77]:
expr_real = sym.re(expr)
expr_real

sqrt(2)*c1*x*cos(t/c**2)*Abs(c) + 2*c2*x*y + cos(t + t/c**2)*Abs(c)

Extract IMAGINARY part

In [78]:
expr_im = sym.im(expr)
expr_im

sqrt(2)*c1*x*sin(t/c**2)*Abs(c) + sin(t + t/c**2)*Abs(c)

SOLVE $x$ for nodal point

In [79]:
x_nodal = sym.solve(expr_im, x)[0]
x_nodal

-sqrt(2)*sin(t + t/c**2)/(2*c1*sin(t/c**2))

In [80]:
expr_real = expr_real.subs(x, x_nodal).simplify()
expr_real

-(c1*sin(t)*Abs(c) + sqrt(2)*c2*y*sin(t + t/c**2))/(c1*sin(t/c**2))

SOLVE $y$ for nodal point

In [81]:
y_nodal = sym.solve(expr_real, y)[0]
y_nodal

-sqrt(2)*c1*sin(t)*Abs(c)/(2*c2*sin(t + t/c**2))

# Bohmian Equations

In [106]:
from sympy.vector import CoordSys3D, gradient
r = CoordSys3D('r')

In [127]:
Psi = Psi.subs([(x, r.x), (y, r.y)])

In [129]:
Del_Psi = gradient(Psi)
Del_Psi

(-r.x*(2*r.x*r.y*c2 + sqrt(2)*r.x*c1*exp(I*t/c**2)*Abs(c) + exp(I*t)*exp(I*t/c**2)*Abs(c))*exp(-r.x**2/2)*exp(-r.y**2/(2*c**2))*exp(-3*I*t/2)*exp(-3*I*t/(2*c**2))/(sqrt(pi)*Abs(c)**(3/2)) + (2*r.y*c2 + sqrt(2)*c1*exp(I*t/c**2)*Abs(c))*exp(-r.x**2/2)*exp(-r.y**2/(2*c**2))*exp(-3*I*t/2)*exp(-3*I*t/(2*c**2))/(sqrt(pi)*Abs(c)**(3/2)))*r.i + (2*r.x*c2*exp(-r.x**2/2)*exp(-r.y**2/(2*c**2))*exp(-3*I*t/2)*exp(-3*I*t/(2*c**2))/(sqrt(pi)*Abs(c)**(3/2)) - r.y*(2*r.x*r.y*c2 + sqrt(2)*r.x*c1*exp(I*t/c**2)*Abs(c) + exp(I*t)*exp(I*t/c**2)*Abs(c))*exp(-r.x**2/2)*exp(-r.y**2/(2*c**2))*exp(-3*I*t/2)*exp(-3*I*t/(2*c**2))/(sqrt(pi)*c**2*Abs(c)**(3/2)))*r.j

In [132]:
expr = Del_Psi/Psi
expr

(sqrt(pi)*(-r.x*(2*r.x*r.y*c2 + sqrt(2)*r.x*c1*exp(I*t/c**2)*Abs(c) + exp(I*t)*exp(I*t/c**2)*Abs(c))*exp(-r.x**2/2)*exp(-r.y**2/(2*c**2))*exp(-3*I*t/2)*exp(-3*I*t/(2*c**2))/(sqrt(pi)*Abs(c)**(3/2)) + (2*r.y*c2 + sqrt(2)*c1*exp(I*t/c**2)*Abs(c))*exp(-r.x**2/2)*exp(-r.y**2/(2*c**2))*exp(-3*I*t/2)*exp(-3*I*t/(2*c**2))/(sqrt(pi)*Abs(c)**(3/2)))*exp(r.x**2/2)*exp(r.y**2/(2*c**2))*exp(3*I*t/2)*exp(3*I*t/(2*c**2))*Abs(c)**(3/2)/(2*r.x*r.y*c2 + sqrt(2)*r.x*c1*exp(I*t/c**2)*Abs(c) + exp(I*t)*exp(I*t/c**2)*Abs(c)))*r.i + (sqrt(pi)*(2*r.x*c2*exp(-r.x**2/2)*exp(-r.y**2/(2*c**2))*exp(-3*I*t/2)*exp(-3*I*t/(2*c**2))/(sqrt(pi)*Abs(c)**(3/2)) - r.y*(2*r.x*r.y*c2 + sqrt(2)*r.x*c1*exp(I*t/c**2)*Abs(c) + exp(I*t)*exp(I*t/c**2)*Abs(c))*exp(-r.x**2/2)*exp(-r.y**2/(2*c**2))*exp(-3*I*t/2)*exp(-3*I*t/(2*c**2))/(sqrt(pi)*c**2*Abs(c)**(3/2)))*exp(r.x**2/2)*exp(r.y**2/(2*c**2))*exp(3*I*t/2)*exp(3*I*t/(2*c**2))*Abs(c)**(3/2)/(2*r.x*r.y*c2 + sqrt(2)*r.x*c1*exp(I*t/c**2)*Abs(c) + exp(I*t)*exp(I*t/c**2)*Abs(c)))*r.j

In [140]:
expr.args[0].simplify()

((-r.x*(2*r.x*r.y*c2 + sqrt(2)*r.x*c1*exp(I*t/c**2)*Abs(c) + exp(I*t*(1 + c**(-2)))*Abs(c)) + 2*r.y*c2 + sqrt(2)*c1*exp(I*t/c**2)*Abs(c))/(2*r.x*r.y*c2 + sqrt(2)*r.x*c1*exp(I*t/c**2)*Abs(c) + exp(I*t + I*t/c**2)*Abs(c)))*r.i

In [141]:
expr.args[1].simplify()

((-2*r.x*r.y**2*c2 - sqrt(2)*r.x*r.y*c1*exp(I*t/c**2)*Abs(c) + 2*r.x*c**2*c2 - r.y*exp(I*t + I*t/c**2)*Abs(c))/(c**2*(2*r.x*r.y*c2 + sqrt(2)*r.x*c1*exp(I*t/c**2)*Abs(c) + exp(I*t + I*t/c**2)*Abs(c))))*r.j

CONSIDER THE FORMULA BELOW WHEN COMPUTING $$Im\{ \frac{\nabla \Psi}{\Psi} \}$$

https://mathworld.wolfram.com/ComplexDivision.html